In [1]:
import pandas as pd                  
import unicodedata                  
import requests                     
import re                           
from datasets import load_dataset   

In [2]:
# Funkcia na odstránenie diakritiky
def remove_diacritics(text):
    return ''.join(
        c for c in unicodedata.normalize('NFKD', text)
        if not unicodedata.combining(c)
    )

In [3]:
# Funkcia na predspracovanie textov (bez zmeny veľkosti písmen)
def preprocess(texts, remove_diacritics_flag=True, remove_stopwords_flag=False):
    
    # Odstránenie špeciálnych znakov, ponechanie slovenských písmen
    texts = texts.str.replace(r'[^a-zA-ZáäčďéíľĺňóôŕšťúýžÁÄČĎÉÍĽĹŇÓÔŔŠŤÚÝŽ\s]', '', regex=True)

    # Odstránenie diakritiky (ak je zvolené)
    if remove_diacritics_flag:
        texts = texts.apply(remove_diacritics)

    # Odstránenie stop-slov (s ohľadom na veľkosť a diakritiku)
    if remove_stopwords_flag:
        texts = texts.apply(lambda x: ' '.join([
            word for word in x.split()
            if word.lower() not in stopwords_with_diacritics
            and remove_diacritics(word).lower() not in ascii_stopwords
        ]))

    return texts

In [4]:
# Stiahnutie stop-slov zo slovenského repozitára
url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-sk/master/stopwords-sk.txt"
response = requests.get(url)
original_stopwords = response.text.strip().split('\n')

# Vytvorenie dvoch množín: s diakritikou a bez diakritiky
ascii_stopwords = set(remove_diacritics(word.strip()) for word in original_stopwords if word.strip())
stopwords_with_diacritics = set(word.strip().lower() for word in original_stopwords if word.strip())

In [5]:
# Načítanie trénovacej časti datasetu TUKE-KEMT/toxic-sk
dataset = load_dataset('TUKE-KEMT/toxic-sk', split='train')
df = pd.DataFrame(dataset)

# Odstránenie prázdnych hodnôt a premenovanie stĺpca pre čitateľnosť
df = df.dropna()
df.rename(columns={'text': 'comments'}, inplace=True)

In [6]:
# Čistenie komentárov. SlovakBERT podporuje diakritiku, preto ju ponechávame
df['processed_comments'] = preprocess(
    df['comments'],
    remove_diacritics_flag=False,
    remove_stopwords_flag=True
)

In [7]:
# Ukážka niekoľkých pôvodných a predspracovaných komentárov
print("\nUkážka čistenia komentárov:")
print(df[['comments', 'processed_comments']].head(10))


Ukážka čistenia komentárov:
                                            comments  \
0                                       Ďakujeme Vám   
1  "Chcú urobiť výsledok testu a aké sú náklady o...   
2                    chvála bohu mladý chcú tiež žiť   
3  nedozvedeli sme sa nič no ale čo by sme sa aj ...   
4    veľká väčšina s vami nesúhlasí to je váš názor.   
5                   uz ti preplo uplne? jebo drbnutý   
6  informovať politika inej krajiny (akejkolvek) ...   
7                    či sa nehanbíš tu dávať statusy   
8  ved ty vies vsetky len osierat ty mas asi mies...   
9                                 gratulujem a verim   

                                  processed_comments  
0                                           Ďakujeme  
1  Chcú urobiť výsledok testu náklady obchodu obc...  
2                         chvála bohu mladý chcú žiť  
3                 nedozvedeli mohli dozvedieť zdroja  
4                      veľká väčšina nesúhlasí názor  
5                       

In [8]:
# Výber potrebných stĺpcov a premenovanie pre konzistenciu
df_out = df[['processed_comments', 'label']]
df_out.columns = ['text', 'label']

# Uloženie do CSV
df_out.to_csv('toxic_sk_for_slovakbert.csv', index=False)

print("\nDataset uložený ako 'toxic_sk_for_slovakbert.csv'")



Dataset uložený ako 'toxic_sk_for_slovakbert.csv'
